# RNN from scratch

In [6]:
import glob
import torch
from importlib import reload

# %load_ext autoreload
# %autoreload 2
import utils
reload(utils)
from utils.files import *

# Data preparation

In [7]:
article_paths = glob.glob("data/articles/*.txt")

corpus = [
    read_file(article_path) for article_path in article_paths
]

print(len(corpus))
print(corpus[0][:100])

1085
1075 Helina, provisional designation 1926 SC, is a stony Eos asteroid from the outer regions of the 


In [37]:
full_text = ' '.join(corpus)

full_text = ' '.join(c for c in full_text if ord(c) < 128)

unique_characters = sorted(list(set(full_text)))

input_size = len(unique_characters)
print(f"Vocab size: {input_size}")

Vocab size: 95


In [38]:
token_to_index = {ch: i for i, ch in enumerate(unique_characters)}
index_to_token = {i: ch for i, ch in enumerate(unique_characters)}

In [ ]:
seq_length = 15
sequences = []
targets = []
for i in range(len(full_text) - seq_length - 1):
    seq = full_text[i:i + seq_length]
    target = full_text[i + seq_length + 1]
    sequences.append(seq)
    targets.append(target)

print(targets[0])
print(sequences[0])

ValueError: too many dimensions 'str'